In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Load the dataset (Ensure you have the dataset in the correct folder)
train_dir = 'path_to_train_data'
validation_dir = 'path_to_validation_data'

# Image Data Generator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# 🔹 1. Define the Vanilla CNN Model
def vanilla_cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Instantiate the Vanilla CNN model
vanilla_model = vanilla_cnn_model()

# 🔹 2. Define the Fine-Tuned VGG16 Model
def vgg16_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    base_model.trainable = False  # Freeze the layers of VGG16
    
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Instantiate the VGG16 model
vgg_model = vgg16_model()

# 🔹 3. Define Callbacks (for saving best model)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')

# 🔹 4. Train the Vanilla CNN Model
vanilla_history = vanilla_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[checkpoint]
)

# 🔹 5. Train the Fine-Tuned VGG16 Model
vgg_history = vgg_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[checkpoint]
)

# 🔹 6. Load the best models (the best ones saved during training)
vanilla_model.load_weights('best_model.h5')
vgg_model.load_weights('best_model.h5')

# 🔹 7. Evaluate the Models
vanilla_loss, vanilla_acc = vanilla_model.evaluate(validation_generator)
vgg_loss, vgg_acc = vgg_model.evaluate(validation_generator)

print(f"Vanilla CNN Accuracy: {vanilla_acc}")
print(f"VGG16 Fine-Tuned Accuracy: {vgg_acc}")

# 🔹 8. Confusion Matrix and Classification Report for Vanilla CNN
y_true = validation_generator.classes
y_pred_vanilla = vanilla_model.predict(validation_generator)
y_pred_vanilla = (y_pred_vanilla > 0.5).astype("int32")

cm_vanilla = confusion_matrix(y_true, y_pred_vanilla)
print("Vanilla CNN Classification Report:")
print(classification_report(y_true, y_pred_vanilla))

# Plot confusion matrix
sns.heatmap(cm_vanilla, annot=True, fmt="d", cmap='Blues', xticklabels=['Cat', 'Dog'], yticklabels=['Cat', 'Dog'])
plt.title("Confusion Matrix - Vanilla CNN")
plt.show()

# 🔹 9. Confusion Matrix and Classification Report for VGG16 Fine-Tuned
y_pred_vgg = vgg_model.predict(validation_generator)
y_pred_vgg = (y_pred_vgg > 0.5).astype("int32")

cm_vgg = confusion_matrix(y_true, y_pred_vgg)
print("VGG16 Fine-Tuned Classification Report:")
print(classification_report(y_true, y_pred_vgg))

# Plot confusion matrix
sns.heatmap(cm_vgg, annot=True, fmt="d", cmap='Blues', xticklabels=['Cat', 'Dog'], yticklabels=['Cat', 'Dog'])
plt.title("Confusion Matrix - VGG16 Fine-Tuned")
plt.show()

# 🔹 10. Precision-Recall Curve
from sklearn.metrics import precision_recall_curve
precision_vanilla, recall_vanilla, _ = precision_recall_curve(y_true, vanilla_model.predict(validation_generator))
precision_vgg, recall_vgg, _ = precision_recall_curve(y_true, vgg_model.predict(validation_generator))

plt.figure(figsize=(8,6))
plt.plot(recall_vanilla, precision_vanilla, label='Vanilla CNN')
plt.plot(recall_vgg, precision_vgg, label='VGG16 Fine-Tuned')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()

# 🔹 11. Analyze Misclassified Examples
def plot_misclassified_images(model, data_generator):
    # Get misclassified images
    misclassified_idx = []
    for i, (x_batch, y_batch) in enumerate(data_generator):
        y_pred_batch = (model.predict(x_batch) > 0.5).astype("int32")
        misclassified_idx.extend(np.where(y_batch != y_pred_batch)[0])
        if len(misclassified_idx) >= 5:  # Limit to first 5 misclassified images
            break

    # Plot misclassified images
    plt.figure(figsize=(10, 10))
    for i, idx in enumerate(misclassified_idx[:5]):
        plt.subplot(1, 5, i + 1)
        plt.imshow(x_batch[idx])
        plt.title(f"True: {['Cat', 'Dog'][y_batch[idx]]}, Pred: {['Cat', 'Dog'][y_pred_batch[idx]]}")
        plt.axis('off')
    plt.show()

# Display misclassified images for both models
print("Misclassified examples for Vanilla CNN:")
plot_misclassified_images(vanilla_model, validation_generator)

print("Misclassified examples for VGG16 Fine-Tuned:")
plot_misclassified_images(vgg_model, validation_generator)

# 🔹 12. Conclusion
print("Conclusion:")
# Summarize your findings based on the results above.
